# Pipelines de treinamento: LogisticRegression

Esse notebook contém as pipelines de treinamento usadas na obtenção do melhor modelo classificador do problema desenvolvido no EP1.

O modelo NaiveBayes aceita diversas parametrizações diferentes - veja documentação: https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB - aqui exploramos duas: *alpha* e *force-alpha*. 

As pipelines treinadas são estratificadas em **features + MultinomialNB:force-alpha**.  
Isto quer dizer: existe uma pipeline para cada par de *feature* e *force-alpha* aqui explorados.  

As **features** que exploramos são:  

1. Bag of Words  
2. TF/TF-IDF  
3. Word NGrams  
4. Char NGrams  

E os **valores de alpha** de *MultinomialNB* que exploramos aqui são *True* e *False*.  

Você pode encontrar uma execução já parametrizada do melhor modelo encontrado por essas pipelines no notebook model.ipynb, **que é nossa versão de entrega do EP1**.

## Bootstrap Imports

In [1]:
import pandas as pd
import numpy as np

import os
import sys
from pathlib import Path

### Teste de importação: Lib.utils do projeto

In [2]:
filedir = Path(os.getcwd())
base_path = filedir.resolve().parents[3]
sys.path.append(str(base_path))

from Lib.utils import printhello
printhello()

HELLO!


## Configura variáveis de execução

In [3]:
sep = ";"
dec = ","
quotech = "\""
encoding = "utf8"


EP_dir = "EP1"
CSV_input_name = "train_complexo_simples.csv"
path_to_archive = f"../../../../Traindata/{EP_dir}/{CSV_input_name}"


do_print = True
if do_print:
    print(f"Path to csv input is:  {path_to_archive}")

Path to csv input is:  ../../../../Traindata/EP1/train_complexo_simples.csv


### Configure variáveis de reprodutibilidade

In [4]:
random_state = 12345

### Lista de melhores modelos

In [5]:
best_models_list = []

## Pré-tratamento de dados

### Importar dados do csv

In [6]:
df = pd.read_csv(path_to_archive, na_values=['na'],
sep=sep,
decimal=dec,
quotechar=quotech,
encoding=encoding,
encoding_errors='strict')
print(df.shape)
print(df.columns)

(33422, 2)
Index(['text', 'style'], dtype='object')


### Embaralhamento dos dados

O .csv de entrada tem alto ordenamento dos inputs por classe. Carregá-los dessa maneira nos modelos p/ treinamento introduz viés, então é preciso embaralhar os dados para garantir randomicidade. 
As classes em sklearn.model_selection - como a StratfiedKFold usada mais a frente - implementam parâmetro shuffle="", que pode ser passado como True para embaralhar mais os dados.

Note que é importante também garantir a reprodutibilidade do embaralhamento, especificando um valor hardcoded (Neste caso random_state=100)

In [7]:
print("Shape antes do shuffle:", df.shape)

df = df.sample(frac=1, random_state=100).reset_index(drop=True)

print("Shape depois do shuffle:", df.shape)

Shape antes do shuffle: (33422, 2)
Shape depois do shuffle: (33422, 2)


# Pipelines

## Imports

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import ParameterGrid

## Disable warnings

In [9]:
import warnings
from sklearn.exceptions import ConvergenceWarning

# Ignora warnings de ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Ignora UserWarnings específicos de l1_ratio etc
warnings.filterwarnings("ignore", category=UserWarning)

## Feature: Bag of words 

#### Definição da Pipeline

In [10]:
BoW_pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('kbest', SelectKBest(score_func=chi2)),
    ('classifier', MultinomialNB()),
]) 

#### Force-alpha=True

In [11]:
#[x for x in range(10, 10000, 5)] + ['all']
BoW_parameters_T = {
    'kbest__k': [10, 20, 30, 40, 50, 100, 750, 4000, 10000, 'all'],
    'classifier__force_alpha': [True],
    'classifier__alpha': [1.0, 0.5, 0.1],
}


BoW_classifier_T = GridSearchCV(BoW_pipeline, BoW_parameters_T, 
                                       cv=10, n_jobs=-1, scoring="accuracy", verbose=1, error_score = np.nan)
BoW_classifier_T.fit(df["text"].fillna(""), df["style"].values)

Fitting 10 folds for each of 30 candidates, totalling 300 fits


,estimator,Pipeline(step...inomialNB())])
,param_grid,"{'classifier__alpha': [1.0, 0.5, ...], 'classifier__force_alpha': [True], 'kbest__k': [10, 20, ...]}"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,10
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,input,'content'


In [12]:
print("Melhor acurácia média T:", BoW_classifier_T.best_score_)
print("Melhores parâmetros T:", BoW_classifier_T.best_params_)

best_models_list.append({
    "features": "BoW",
    "force-alpha": True,
    "accuracy": BoW_classifier_T.best_score_,
    "params": BoW_classifier_T.best_params_,
})

Melhor acurácia média T: 0.8234398968306096
Melhores parâmetros T: {'classifier__alpha': 0.5, 'classifier__force_alpha': True, 'kbest__k': 10000}


#### Force-alpha=False

In [13]:
BoW_parameters_F = {
    'kbest__k': [10, 20, 30, 40, 50, 100, 750, 4000, 10000, 'all'],
    'classifier__force_alpha': [False],
    'classifier__alpha': [1e-10, 1e-11, 1e-12, 1e-13, 0],
}


BoW_classifier_F = GridSearchCV(BoW_pipeline, BoW_parameters_F, 
                                       cv=10, n_jobs=2, scoring="accuracy", verbose=1, error_score = np.nan)
BoW_classifier_F.fit(df["text"].fillna(""), df["style"].values)

Fitting 10 folds for each of 50 candidates, totalling 500 fits


,estimator,Pipeline(step...inomialNB())])
,param_grid,"{'classifier__alpha': [1e-10, 1e-11, ...], 'classifier__force_alpha': [False], 'kbest__k': [10, 20, ...]}"
,scoring,'accuracy'
,n_jobs,2
,refit,True
,cv,10
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,input,'content'


In [14]:
print("Melhor acurácia média F:", BoW_classifier_F.best_score_)
print("Melhores parâmetros F:", BoW_classifier_F.best_params_)

best_models_list.append({
    "features": "BoW",
    "force-alpha": False,
    "accuracy": BoW_classifier_F.best_score_,
    "params": BoW_classifier_F.best_params_,
})

Melhor acurácia média F: 0.8175453840952798
Melhores parâmetros F: {'classifier__alpha': 1e-10, 'classifier__force_alpha': False, 'kbest__k': 4000}


### TF e TF-IDF

#### Definição da Pipeline

In [15]:
TF_pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('kbest', SelectKBest(score_func=chi2)),
    ('classifier', MultinomialNB()),
])

#### Force-alpha=True

In [16]:
TF_parameters_T = {
    'tfidf__use_idf': [True, False],
    'kbest__k': [10, 20, 30, 40, 50, 100, 750, 4000, 10000, 'all'],
    'classifier__force_alpha': [True],
    'classifier__alpha': [1.0, 0.5, 0.1],
}

TF_classifier_T = GridSearchCV(TF_pipeline, TF_parameters_T, 
                                       cv=10, n_jobs=-1, scoring="accuracy", verbose=1, error_score = np.nan)
TF_classifier_T.fit(df["text"].fillna(""), df["style"].values)

Fitting 10 folds for each of 60 candidates, totalling 600 fits


,estimator,Pipeline(step...inomialNB())])
,param_grid,"{'classifier__alpha': [1.0, 0.5, ...], 'classifier__force_alpha': [True], 'kbest__k': [10, 20, ...], 'tfidf__use_idf': [True, False]}"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,10
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,input,'content'


In [17]:
print("Melhor acurácia média:", TF_classifier_T.best_score_)
print("Melhores parâmetros:", TF_classifier_T.best_params_)

best_models_list.append({
    "features": "TF",
    "force-alpha": True,
    "accuracy": TF_classifier_T.best_score_,
    "params": TF_classifier_T.best_params_,
})

Melhor acurácia média: 0.8206572483782667
Melhores parâmetros: {'classifier__alpha': 0.1, 'classifier__force_alpha': True, 'kbest__k': 10000, 'tfidf__use_idf': False}


#### Force-alpha=False

In [18]:
TF_parameters_F = {
    'tfidf__use_idf': [True, False],
    'kbest__k': [10, 20, 30, 40, 50, 100, 750, 4000, 10000, 'all'],
    'classifier__force_alpha': [True],
    'classifier__alpha': [1.0, 0.5, 0.1],
}

TF_classifier_F = GridSearchCV(TF_pipeline, TF_parameters_F, 
                                       cv=10, n_jobs=-1, scoring="accuracy", verbose=1, error_score = np.nan)
TF_classifier_F.fit(df["text"].fillna(""), df["style"].values)

Fitting 10 folds for each of 60 candidates, totalling 600 fits


,estimator,Pipeline(step...inomialNB())])
,param_grid,"{'classifier__alpha': [1.0, 0.5, ...], 'classifier__force_alpha': [True], 'kbest__k': [10, 20, ...], 'tfidf__use_idf': [True, False]}"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,10
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,input,'content'


In [19]:
print("Melhor acurácia média:", TF_classifier_F.best_score_)
print("Melhores parâmetros:", TF_classifier_F.best_params_)

best_models_list.append({
    "features": "TF",
    "force-alpha": False,
    "accuracy": TF_classifier_F.best_score_,
    "params": TF_classifier_F.best_params_,
})

Melhor acurácia média: 0.8206572483782667
Melhores parâmetros: {'classifier__alpha': 0.1, 'classifier__force_alpha': True, 'kbest__k': 10000, 'tfidf__use_idf': False}


## N-grams

#### Definição da Pipeline

In [20]:
NGram_pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('kbest', SelectKBest(score_func=chi2)),
    ('classifier', MultinomialNB()),
]) 

### WORD NGRAMS

#### Force-alpha=True

In [21]:
WORD_NGram_parameters_T = { 
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'vect__analyzer': ["word"],
    'kbest__k': [10, 20, 30, 40, 50, 100, 750, 4000, 10000, 'all'],
    'classifier__force_alpha': [True],
    'classifier__alpha': [1.0, 0.5, 0.1],
}


WORD_NGram_classifier_T = GridSearchCV(NGram_pipeline, WORD_NGram_parameters_T, 
                                       cv=10, n_jobs=-1, scoring="accuracy", verbose=1, error_score = np.nan)
WORD_NGram_classifier_T.fit(df["text"].fillna(""), df["style"].values)

Fitting 10 folds for each of 90 candidates, totalling 900 fits


,estimator,Pipeline(step...inomialNB())])
,param_grid,"{'classifier__alpha': [1.0, 0.5, ...], 'classifier__force_alpha': [True], 'kbest__k': [10, 20, ...], 'vect__analyzer': ['word'], ...}"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,10
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,input,'content'


In [22]:
print("Melhor acurácia média:", WORD_NGram_classifier_T.best_score_)
print("Melhores parâmetros:", WORD_NGram_classifier_T.best_params_)

best_models_list.append({
    "features": "WORD-NGram",
    "force-alpha": True,
    "accuracy": WORD_NGram_classifier_T.best_score_,
    "params": WORD_NGram_classifier_T.best_params_,
})

Melhor acurácia média: 0.8430078177235746
Melhores parâmetros: {'classifier__alpha': 0.5, 'classifier__force_alpha': True, 'kbest__k': 10000, 'vect__analyzer': 'word', 'vect__ngram_range': (1, 2)}


#### Force-alpha=False

In [23]:
WORD_NGram_parameters_F = { 
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'vect__analyzer': ["word"],
    'kbest__k': [10, 20, 30, 40, 50, 100, 750, 4000, 10000, 'all'],
    'classifier__force_alpha': [False],
    'classifier__alpha': [1e-10, 1e-11, 1e-12, 1e-13, 0],
}


WORD_NGram_classifier_F = GridSearchCV(NGram_pipeline, WORD_NGram_parameters_F, 
                                       cv=10, n_jobs=-1, scoring="accuracy", verbose=1, error_score = np.nan)
WORD_NGram_classifier_F.fit(df["text"].fillna(""), df["style"].values)

Fitting 10 folds for each of 150 candidates, totalling 1500 fits


,estimator,Pipeline(step...inomialNB())])
,param_grid,"{'classifier__alpha': [1e-10, 1e-11, ...], 'classifier__force_alpha': [False], 'kbest__k': [10, 20, ...], 'vect__analyzer': ['word'], ...}"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,10
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,input,'content'


In [24]:
print("Melhor acurácia média:", WORD_NGram_classifier_F.best_score_)
print("Melhores parâmetros:", WORD_NGram_classifier_F.best_params_)

best_models_list.append({
    "features": "WORD-NGram",
    "force-alpha": False,
    "accuracy": WORD_NGram_classifier_F.best_score_,
    "params": WORD_NGram_classifier_F.best_params_,
})

Melhor acurácia média: 0.8326551474691082
Melhores parâmetros: {'classifier__alpha': 1e-10, 'classifier__force_alpha': False, 'kbest__k': 10000, 'vect__analyzer': 'word', 'vect__ngram_range': (1, 2)}


### CHAR NGram

#### Force-alpha=True

In [25]:
CHAR_NGram_parameters_T = { 
    'vect__ngram_range': [(1, 1), (2, 2), (3, 3), (4, 4)],
    'vect__analyzer': ["char", "char_wb"],
    'kbest__k': [10, 20, 30, 40, 50, 100, 750, 4000, 10000, 'all'],
    'classifier__force_alpha': [True],
    'classifier__alpha': [1.0, 0.5, 0.1],
}


CHAR_NGram_classifier_T = GridSearchCV(NGram_pipeline, CHAR_NGram_parameters_T, 
                                       cv=10, n_jobs=-1, scoring="accuracy", verbose=1, error_score = np.nan)
CHAR_NGram_classifier_T.fit(df["text"].fillna(""), df["style"].values)

Fitting 10 folds for each of 240 candidates, totalling 2400 fits


,estimator,Pipeline(step...inomialNB())])
,param_grid,"{'classifier__alpha': [1.0, 0.5, ...], 'classifier__force_alpha': [True], 'kbest__k': [10, 20, ...], 'vect__analyzer': ['char', 'char_wb'], ...}"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,10
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,input,'content'


In [26]:
print("Melhor acurácia média:", CHAR_NGram_classifier_T.best_score_)
print("Melhores parâmetros:", CHAR_NGram_classifier_T.best_params_)

best_models_list.append({
    "features": "CHAR-NGram",
    "force-alpha": True,
    "accuracy": CHAR_NGram_classifier_T.best_score_,
    "params": CHAR_NGram_classifier_T.best_params_,
})

Melhor acurácia média: 0.8652385371471208
Melhores parâmetros: {'classifier__alpha': 0.1, 'classifier__force_alpha': True, 'kbest__k': 10000, 'vect__analyzer': 'char', 'vect__ngram_range': (4, 4)}


#### Force-alpha=False

In [27]:
CHAR_NGram_parameters_F = { 
    'vect__ngram_range': [(1, 1), (2, 2), (3, 3), (4, 4)],
    'vect__analyzer': ["char", "char_wb"],
    'kbest__k': [10, 20, 30, 40, 50, 100, 750, 4000, 10000, 'all'],
    'classifier__force_alpha': [False],
    'classifier__alpha': [1e-10, 1e-11, 1e-12, 1e-13, 0],
}


CHAR_NGram_classifier_F = GridSearchCV(NGram_pipeline, CHAR_NGram_parameters_F, 
                                       cv=10, n_jobs=-1, scoring="accuracy", verbose=1, error_score = np.nan)
CHAR_NGram_classifier_F.fit(df["text"].fillna(""), df["style"].values)

Fitting 10 folds for each of 400 candidates, totalling 4000 fits


,estimator,Pipeline(step...inomialNB())])
,param_grid,"{'classifier__alpha': [1e-10, 1e-11, ...], 'classifier__force_alpha': [False], 'kbest__k': [10, 20, ...], 'vect__analyzer': ['char', 'char_wb'], ...}"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,10
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,input,'content'


In [28]:
print("Melhor acurácia média:", CHAR_NGram_classifier_F.best_score_)
print("Melhores parâmetros:", CHAR_NGram_classifier_F.best_params_)

best_models_list.append({
    "features": "CHAR-NGram",
    "force-alpha": False,
    "accuracy": CHAR_NGram_classifier_F.best_score_,
    "params": CHAR_NGram_classifier_F.best_params_,
})

Melhor acurácia média: 0.8616779740905771
Melhores parâmetros: {'classifier__alpha': 1e-10, 'classifier__force_alpha': False, 'kbest__k': 10000, 'vect__analyzer': 'char', 'vect__ngram_range': (4, 4)}


## Seleciona melhores parâmetros

In [29]:
best_score = -1
best = 0
for idx, candidate in enumerate(best_models_list):
    if candidate["accuracy"] > best_score:
        best = idx
        best_score = candidate["accuracy"]

print(f"O melhor classificador encontrado pelas pipelines é -->    feature={best_models_list[best]["features"]} + force-alpha={best_models_list[best]["force-alpha"]}\n")
print(f"Melhor acucácia encontrada:  {best_models_list[best]['accuracy']}")
print(f"Melhores parametros encontrados:  {best_models_list[best]['params']}")


O melhor classificador encontrado pelas pipelines é -->    feature=CHAR-NGram + force-alpha=True

Melhor acucácia encontrada:  0.8652385371471208
Melhores parametros encontrados:  {'classifier__alpha': 0.1, 'classifier__force_alpha': True, 'kbest__k': 10000, 'vect__analyzer': 'char', 'vect__ngram_range': (4, 4)}
